In [1]:
import json 
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.metrics import jaccard_score
import sys 
from tqdm.notebook import tqdm
import sklearn.feature_extraction.text as ft 



In [2]:
sys.path.insert(0, '../')
import src.preprocessing.text_preprocessing as tp

In [3]:
plt.rcParams['text.usetex'] = False

In [24]:
FOLDER = 'brown_remove'
base_path = '../results/'
path = f'{base_path}{FOLDER}/'
full_paths = [f'{path}{file}' for file in os.listdir(path)]


In [25]:
files = {}
for p in full_paths:
    name = p.split('/')[-1]
    split = name.split('_')
    method = '_'.join(split[1:-1])
    n_words = split[-1].split('.')[0]
    with open(p, 'r') as file:
        if method not in files:
            files[method] = {}
        files[method][n_words] = json.load(file)
    

In [26]:
methods_renames = {
    'shap': 'SHAP',
    'term_strength': 'TS',
    'trl': 'TRL',
    'eccd': 'ECCD',
    'mutual_information': 'MI',
    'chi2': 'chi2',
    'tfidf': 'TF-IDF',
    'linear_measure_5': 'LM',
    'lfs': 'LFS',
    'baseline': 'Baseline'
}

In [27]:
tqdm.pandas()
df = pd.read_csv('../data/brown_corpus/brown_corpus.csv', sep=';')
df = df.fillna('')
df = df.astype('str')
df['Label'] = df['Label'].astype('category')
df['Label'] = df['Label'].cat.codes
df['Text'] = df['Text'].apply(tp.normalize_text)

count_vectorizer = ft.CountVectorizer()
count_vectorizer.fit(df['Text'])
vocabulary = count_vectorizer.get_feature_names_out()
print(len(vocabulary))

35366


In [35]:
TEST_TRAIN = 'test'

In [29]:
def get_extractor_timings(files: dict) -> pd.DataFrame:
    methods = files.keys()
    df = pd.DataFrame(columns=methods)
    n_words_list = set()
    method_list = set()
    for method, words_dict in files.items():
        method_list.add(method)
        for n_words, info in words_dict.items():
                df.loc[n_words, method] = info['timing']['extractor_fit']
                n_words_list.add(n_words)
                
    df.index = df.index.astype('int')
    df = df.sort_index()
    df = df.apply(pd.to_timedelta)
    for col in df.columns:
        df[col] = df[col].dt.total_seconds()
    return df, method_list, n_words_list

def plot_extractors_timings(df) -> plt.figure:
    df = df.copy(deep=True)
    df = df.rename(columns=methods_renames)
    axes = df.plot(logy=True, logx=True, figsize=(16, 10))
    axes.set_xlabel('# selected words')
    axes.set_ylabel('runtime (sec)')
    # axes.set_label('runtime (sec)')
    axes.legend(loc="lower right")

    # plt.title('Feature extractor runtimes vs number of the selected words.')
    plt.savefig('../figures/brown/brown_extractor_runtime_vs_n_words.pdf', bbox_inches="tight")
    plt.close()

def get_selected_words_per_extractor_per_n_words(files: dict, vocabulary, n_words_list, method_list):
    df_dict = {}

    for n_words in n_words_list:
        df_dict[n_words] = pd.DataFrame(index=vocabulary, columns=list(method_list)).fillna(0)

    for method, words_dict in files.items():
        for n_words, info in words_dict.items():
            df_dict[n_words].loc[info['selected_vocabulary'][0], method] = 1
    return df_dict

def get_cross_jaccard_score(df):
    methods = df.columns.tolist()
    jaccard_df = pd.DataFrame(index=methods, columns=methods)
    for method_1 in methods:
        for method_2 in methods:
            jaccard_df.loc[method_1, method_2] = jaccard_score(df[method_1], df[method_2])
    return jaccard_df

def get_similarity_metrics(df_dict):
    correlations_dict = {}
    jaccard_score_dict = {}
    for n_words, df in df_dict.items():
        df_filtered = df.loc[:,(df.sum(axis=0) != 0).values] # remove lfs (or other methods) when they have no values
        correlations_dict[n_words] = df_filtered.corr() 
        jaccard_score_dict[n_words] = get_cross_jaccard_score(df_filtered)
    
    return correlations_dict, jaccard_score_dict

def compare_shap_over_n_words_set_similarity(df_dict: dict, n_words_list, method_list):
    df_comp = pd.DataFrame(columns=method_list, index=n_words_list)
    for n_words, df in df_dict.items():
        cols = df.columns
        df_comp.loc[n_words, cols] = df['shap'][cols]
    df_comp.index = df_comp.index.astype('int')
    df_comp = df_comp.sort_index()
    df_comp = df_comp.drop(columns=['shap'])
    return df_comp

def compare_performance_over_n_words(files, n_words_list, method_list, baseline=None):
    metrics = ['precision', 'recall', 'f1-score']
    cols = []
    for method in method_list:
        for metric in metrics:
            cols.append((method, f'{metric}_mean'))
            cols.append((method, f'{metric}_std'))
    df_metrics = pd.DataFrame(columns=pd.MultiIndex.from_tuples(cols), index=n_words_list)

    for method, words_dict in files.items():
        for n_words, info in words_dict.items():
            for metric in metrics:
                df_metrics.loc[n_words, (method, f'{metric}_mean')] = info[f'classification_report_{TEST_TRAIN}']['macro avg'][f'{metric}_mean']
                df_metrics.loc[n_words, (method, f'{metric}_std')] = info[f'classification_report_{TEST_TRAIN}']['macro avg'][f'{metric}_std']
    if baseline is not None:
        for n_words in n_words_list:
            for metric in metrics:
                df_metrics.loc[n_words, ('baseline',f'{metric}_mean')] = baseline['macro avg'][f'{metric}_mean']
                df_metrics.loc[n_words, ('baseline',f'{metric}_std')] = baseline['macro avg'][f'{metric}_std']

    df_metrics.index = df_metrics.index.astype('int')
    df_metrics = df_metrics.sort_index()
    return df_metrics

def plot_performance_metrics(df_metrics, metrics_to_plot, methods_to_plot):
    for metric in metrics_to_plot:
        fig, ax = plt.subplots(figsize=(9,5))

        for method in methods_to_plot:
            errbar = ax.errorbar(x=df_metrics.index, y=df_metrics.loc[:, (method, f'{metric}_mean')], yerr=df_metrics.loc[:, (method, f'{metric}_std')], label=methods_renames[method], alpha=0.5, linestyle='dashed', fmt='--', barsabove=True, elinewidth=2, linewidth=1)
            errbar[-1][0].set_linestyle(':')
        ax.set_xscale('log')
        ax.set_ylabel('log')
        ax.set_xlabel('# of kept words')
        ax.set_ylabel(f'{metric}')
        ax.legend()
        plt.title(f'Brown corpus {metric} vs # of removed words.')
        plt.savefig(f'../figures/brown/brown_perf_{metric}_remove_{TEST_TRAIN}.pdf')
        plt.close()

In [30]:
df, method_list, n_words_list = get_extractor_timings(files)
# df.to_csv('../figures/csv_outputs/timings/brown_n_word_timings.csv', sep=';')
# plot_extractors_timings(df)

In [31]:
df_dict = get_selected_words_per_extractor_per_n_words(files, vocabulary, n_words_list, method_list)
correlations_dict, jaccard_score_dict = get_similarity_metrics(df_dict)

In [20]:
# for words, m in jaccard_score_dict.items():
#     print(f'{words}: {m.keys()}')

In [32]:
# shap_correlations = compare_shap_over_n_words_set_similarity(correlations_dict, n_words_list, method_list)
shap_jaccard = compare_shap_over_n_words_set_similarity(jaccard_score_dict, n_words_list, method_list)
# shap_correlations.to_csv('../results/tables/brown_correlations.csv', sep=';')
shap_jaccard.to_csv(f'../figures/csv_outputs/jaccard/brown_jaccard_remove_{TEST_TRAIN}.csv', sep=';')

In [33]:
with open('../results/brown_baseline.json', 'r') as file:
    baseline_brown = json.load(file)

In [34]:
df_metrics = compare_performance_over_n_words(files, n_words_list, method_list, baseline=baseline_brown[f'classification_report_{TEST_TRAIN}']) 
df_metrics.to_csv(f'../figures/csv_outputs/metrics/brown_performance_remove_{TEST_TRAIN}.csv', sep=';')

In [217]:
methods = list(method_list)[:]
methods.append('baseline')
plot_performance_metrics(df_metrics, ['recall', 'precision', 'f1-score'], methods)

In [15]:
jaccard_table = pd.read_csv('../results/tables/enron_jaccard.csv', sep=';', index_col=0)
jaccard_table.index.name = 'n_words'
# jaccard_table = jaccard_table.drop(columns='lfs')

In [18]:
with pd.option_context('display.float_format', '{:,.3f}'.format):
    jaccard_table = jaccard_table.rename(columns=methods_renames)
    print(jaccard_table.to_latex(caption='Jaccard score between Shapley values based method and the rest.'))

\begin{table}
\centering
\caption{Jaccard score between Shapley values based method and the rest.}
\begin{tabular}{lrrrrrrr}
\toprule
{} &  ECCD &    MI &   TRL &    LM &    TS &  TF-IDF &  chi2 \\
n\_words &       &       &       &       &       &         &       \\
\midrule
50      & 0.205 & 0.351 & 0.075 & 0.235 & 0.220 &   0.266 & 0.299 \\
100     & 0.235 & 0.351 & 0.047 & 0.282 & 0.266 &   0.282 & 0.351 \\
200     & 0.286 & 0.394 & 0.042 & 0.356 & 0.338 &   0.270 & 0.375 \\
500     & 0.247 & 0.344 & 0.018 & 0.445 & 0.422 &   0.357 & 0.350 \\
1000    & 0.214 & 0.299 & 0.010 & 0.505 & 0.499 &   0.463 & 0.305 \\
3000    & 0.099 & 0.132 & 0.016 & 0.192 & 0.191 &   0.184 & 0.137 \\
5000    & 0.082 & 0.100 & 0.024 & 0.125 & 0.125 &   0.121 & 0.102 \\
10000   & 0.070 & 0.076 & 0.042 & 0.085 & 0.085 &   0.085 & 0.077 \\
15000   & 0.082 & 0.086 & 0.037 & 0.092 & 0.092 &   0.092 & 0.087 \\
25000   & 0.112 & 0.113 & 0.110 & 0.113 & 0.113 &   0.114 & 0.113 \\
\bottomrule
\end{tabular}
\end{ta

In [20]:
jaccard_3000_df = jaccard_score_dict['15000']
jaccard_3000_df = jaccard_3000_df.rename(columns=methods_renames)
jaccard_3000_df = jaccard_3000_df.rename(index=methods_renames)
with pd.option_context('display.float_format', '{:,.3f}'.format):
    print(jaccard_3000_df.to_latex(caption='Jaccard score between the methods at 3000 kept words.'))


\begin{table}
\centering
\caption{Jaccard score between the methods at 3000 kept words.}
\begin{tabular}{lllllllll}
\toprule
{} & TF-IDF &    LM &   TRL &  ECCD &    TS &  SHAP &    MI &  chi2 \\
\midrule
TF-IDF &  1.000 & 0.536 & 0.477 & 0.674 & 0.630 & 0.357 & 0.719 & 0.777 \\
LM     &  0.536 & 1.000 & 0.403 & 0.537 & 0.740 & 0.435 & 0.644 & 0.545 \\
TRL    &  0.477 & 0.403 & 1.000 & 0.352 & 0.477 & 0.348 & 0.532 & 0.399 \\
ECCD   &  0.674 & 0.537 & 0.352 & 1.000 & 0.544 & 0.336 & 0.581 & 0.817 \\
TS     &  0.630 & 0.740 & 0.477 & 0.544 & 1.000 & 0.396 & 0.795 & 0.592 \\
SHAP   &  0.357 & 0.435 & 0.348 & 0.336 & 0.396 & 1.000 & 0.362 & 0.354 \\
MI     &  0.719 & 0.644 & 0.532 & 0.581 & 0.795 & 0.362 & 1.000 & 0.672 \\
chi2   &  0.777 & 0.545 & 0.399 & 0.817 & 0.592 & 0.354 & 0.672 & 1.000 \\
\bottomrule
\end{tabular}
\end{table}

